# Importando Bibliotecas

In [27]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go

from sklearn import metrics

from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics

import itertools

import warnings
warnings.filterwarnings('ignore')

# Lendo os Dados

In [4]:
df = pd.read_csv('../Dados/df_final.csv')
df.head()

,data,vendas,vendas_Outra_Faculdade,vendas_PUCRIO,vendas_UFABC,vendas_UFF,vendas_UFMG,vendas_UFPE,vendas_UFRJ,vendas_UFSC,...,vendas_Natal,vendas_Niteroi,vendas_Outra_Cidade,vendas_Porto_Alegre,vendas_Recife,vendas_Rio_de_Janeiro,vendas_Salvador,vendas_Santo_Andre,vendas_Sao_Carlos,vendas_Sao_Paulo
0,2018-01-01,1819.0,618.0,100.0,17.0,194.0,34.0,71.0,407.0,23.0,...,3.0,79.0,587.0,11.0,51.0,521.0,25.0,9.0,14.0,187.0
1,2018-02-01,2012.0,652.0,98.0,22.0,199.0,34.0,85.0,492.0,28.0,...,2.0,81.0,654.0,11.0,62.0,592.0,23.0,13.0,14.0,205.0
2,2018-03-01,4035.0,926.0,229.0,16.0,357.0,46.0,163.0,1473.0,30.0,...,9.0,171.0,1103.0,14.0,123.0,1544.0,22.0,12.0,29.0,282.0
3,2018-04-01,4305.0,1204.0,189.0,186.0,512.0,96.0,134.0,1098.0,151.0,...,4.0,232.0,1299.0,21.0,105.0,1300.0,38.0,40.0,27.0,359.0
4,2018-05-01,3956.0,1462.0,176.0,130.0,349.0,212.0,158.0,721.0,122.0,...,8.0,155.0,1250.0,20.0,110.0,1065.0,41.0,39.0,31.0,300.0


# Prophet

In [10]:
df_prophet = df[['data', 'vendas']]
df_prophet.rename(columns={
    'data': 'ds',
    'vendas': 'y'
}, inplace=True)

In [11]:
model = Prophet()
model.fit(df_prophet)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [14]:
forecast = model.predict(df_prophet)
forecast.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2018-01-01,3032.788457,-319.805664,1907.750327,3032.788457,3032.788457,-2302.055139,-2302.055139,-2302.055139,-2302.055139,-2302.055139,-2302.055139,0.0,0.0,0.0,730.733318
1,2018-02-01,3198.974597,456.790878,2748.690305,3198.974597,3198.974597,-1617.857728,-1617.857728,-1617.857728,-1617.857728,-1617.857728,-1617.857728,0.0,0.0,0.0,1581.116869
2,2018-03-01,3349.078209,2573.593689,4808.331263,3349.078209,3349.078209,342.371871,342.371871,342.371871,342.371871,342.371871,342.371871,0.0,0.0,0.0,3691.450080
3,2018-04-01,3515.264350,2869.914097,5117.486797,3515.264350,3515.264350,491.257099,491.257099,491.257099,491.257099,491.257099,491.257099,0.0,0.0,0.0,4006.521448
4,2018-05-01,3676.089647,3346.963198,5509.523014,3676.089647,3676.089647,722.671422,722.671422,722.671422,722.671422,722.671422,722.671422,0.0,0.0,0.0,4398.761070


In [24]:
fig = go.Figure(data=[
    go.Scatter(name='Previsto',x=forecast['ds'], y=forecast['yhat'], marker={'color': 'orange'}),
    go.Scatter(name='Real', x=df_prophet['ds'], y=df_prophet['y'], marker={'color': 'gray'})
    ])
fig.update_layout(font=dict(color='black'), title_text='Real vs. Previsto (Prophet)')
fig.show()

In [25]:
mse = metrics.mean_squared_error(df_prophet['y'], forecast['yhat'])
rmse = metrics.mean_squared_error(df_prophet['y'], forecast['yhat'], squared=False)
mad = metrics.mean_absolute_error(df_prophet['y'], forecast['yhat'])
mape = metrics.mean_absolute_percentage_error(df_prophet['y'], forecast['yhat'])

print(f'''
      EQM: {round(mse,2)}
      REQM: {round(rmse,2)}
      MAD: {round(mad,2)}
      MAPE: {round(mape*100,2)}%
      ''')


      EQM: 764752.27
      REQM: 874.5
      MAD: 696.18
      MAPE: 10.57%
      


### Tuning

In [74]:
param_grid = {  
    'changepoint_prior_scale': [0.8, 1.0],
    'seasonality_prior_scale': [0.8, 0.9, 1.0],
    'seasonality_mode': ['additive', 'multiplicative']
}

In [77]:
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

In [78]:
mape = []

for params in all_params:
    m = Prophet(**params).fit(df_prophet) 
    df_cv = cross_validation(m, initial='1460 days', horizon='60 days', period='30 days', parallel="processes")
    df_p = performance_metrics(df_cv, rolling_window=1)
    mape.append(df_p['mape'].values[0])

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 2 forecasts with cutoffs between 2022-01-01 00:00:00 and 2022-01-31 00:00:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x00000244124EEF10>
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 2 forecasts with cutoffs between 2022-01-01 00:00:00 and 2022-01-31 00:00:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x000002441248B220>
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_s

In [79]:
tuning_results = pd.DataFrame(all_params)
tuning_results['mape'] = mape
tuning_results

,changepoint_prior_scale,seasonality_prior_scale,seasonality_mode,mape
0,0.8,0.8,additive,0.080274
1,0.8,0.8,multiplicative,0.358089
2,0.8,0.9,additive,0.073173
3,0.8,0.9,multiplicative,0.347968
4,0.8,1.0,additive,0.080548
5,0.8,1.0,multiplicative,0.360108
6,1.0,0.8,additive,0.036915
7,1.0,0.8,multiplicative,0.244794
8,1.0,0.9,additive,0.037029
9,1.0,0.9,multiplicative,0.239865


In [80]:
tuning_results.sort_values(by='mape', ascending=True)

,changepoint_prior_scale,seasonality_prior_scale,seasonality_mode,mape
6,1.0,0.8,additive,0.036915
8,1.0,0.9,additive,0.037029
10,1.0,1.0,additive,0.037090
2,0.8,0.9,additive,0.073173
0,0.8,0.8,additive,0.080274
4,0.8,1.0,additive,0.080548
9,1.0,0.9,multiplicative,0.239865
7,1.0,0.8,multiplicative,0.244794
11,1.0,1.0,multiplicative,0.246998
3,0.8,0.9,multiplicative,0.347968


In [82]:
best_params = all_params[np.argmin(mape)]
print(np.min(mape))
print(best_params)

0.0369151211521806
{'changepoint_prior_scale': 1.0, 'seasonality_prior_scale': 0.8, 'seasonality_mode': 'additive'}


In [83]:
model = Prophet(
    seasonality_mode=best_params['seasonality_mode'],
    seasonality_prior_scale=best_params['seasonality_prior_scale'],
    changepoint_prior_scale=best_params['changepoint_prior_scale'])

In [84]:
model.fit(df_prophet)
forecast = model.predict(df_prophet)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [85]:
fig = go.Figure(data=[
    go.Scatter(name='Previsto',x=forecast['ds'], y=forecast['yhat'], marker={'color': 'orange'}),
    go.Scatter(name='Real', x=df_prophet['ds'], y=df_prophet['y'], marker={'color': 'gray'})
    ])
fig.update_layout(font=dict(color='black'), title_text='Real vs. Previsto (Prophet)')
fig.show()

In [86]:
mse = metrics.mean_squared_error(df_prophet['y'], forecast['yhat'])
rmse = metrics.mean_squared_error(df_prophet['y'], forecast['yhat'], squared=False)
mad = metrics.mean_absolute_error(df_prophet['y'], forecast['yhat'])
mape = metrics.mean_absolute_percentage_error(df_prophet['y'], forecast['yhat'])

print(f'''
      EQM: {round(mse,2)}
      REQM: {round(rmse,2)}
      MAD: {round(mad,2)}
      MAPE: {round(mape*100,2)}%
      ''')


      EQM: 47189.23
      REQM: 217.23
      MAD: 160.92
      MAPE: 2.76%
      
